<b>Research Proposal  - research question formulation and data exploration</b>
<BR>
Data exploration for a research project conducted about body perception and body acceptance
<BR>
<BR>
<b>Data exploration</b>
<BR>
The first step to get to the heart of the research process is to load the dataset and, once explored the variables, to select the variables considered functional to the analysis process, i.e. the ones that could, hypothetically, play a role in determining body acceptance.
Down here, the R script produced to load some fundamental packages is reported.


In [ ]:
#1 Loading the packages to read the dta file and analyze the dataset
install.packages("GPArotation")
install.packages("survey")
install.packages("broom")
install.packages("psych")
install.packages("tidyverse")
install.packages("haven")
install.packages("dplyr")
install.packages("lm.beta")
install.packages("sjlabelled")

library(GPArotation)
library(survey)
library(broom)
library(psych)
library(tidyverse)
library(haven)
library(dplyr)
library(lm.beta)
#1.1 Loading the package to treat dta dataset and converting the dataset
library(sjlabelled)



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: grid

Loading required package: Matrix

Loading required package: survival


Attaching package: ‘survey’


The following object is masked from ‘package:graphics’:

    dotchart


Warning message in system("timedatectl", intern 

Let's load the data. Since our dataset is in dta format (STATA), we need to use the *read_dta* function.

In [ ]:
#2 Reading the dataset
Bodydataset <- read_dta("Body12.dta")
#3 Creating an unlabeled copy of the dataset
BDS <- as.data.frame(Bodydataset)
BDS <- remove_all_labels(BDS)

<b>Social pressure on body care, gender, department of study and psychological features </b>
<BR>
Once the <b>dataset</b> has been read, we can start the explorative analysis. To begin with, We will explore each questionnaire item that can be considered useful to define a feature that plays a relavant role in predicting the way in which people relate with their own body, and more specifically, with the acceptance of his/her body. 
The first items considered are the ones from the battery B3, that contains the first block of questions concerning body care addressed to the interviewed people. In these battery, there are questions that measure the tendency that the people have to compare their own  appearance to the one of the other people in <b>social</b> contexts.<BR>
In other words, these item seems to be aimed at measuring the impact of the social pressure on the perception of one’s body.
Since all these questions are about the same concept, I hypothesized that they could be considered as components of a unique scale. To prove that, I computed Cronbach's Alpha, reversing the question that measured the hyothesized concept in a negative way (the comparison to other people seen as a "bad" instead of a positive thing). 


In [ ]:
#3Subsetting B3 items
B3 <- subset(BDS, select=(c(B3_1,B3_2,B3_3,B3_4,B3_5)))
#4 Creating reversed B3_4:
B3$B3_4_r<-6-BDS$B3_4
#5 Deleting old B3_4
B3$B3_4<-NULL
#6 Computing Crobach Alpha
install.packages("ltm")
library(ltm)
cronbach.alpha(na.omit(B3))

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘mvtnorm’, ‘expm’, ‘msm’, ‘polycor’


Loading required package: MASS


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: msm

Loading required package: polycor


Attaching package: ‘polycor’


The following object is masked from ‘package:psych’:

    polyserial



Attaching package: ‘ltm’


The following object is masked from ‘package:psych’:

    factor.scores





Cronbach's alpha for the 'na.omit(B3)' data-set

Items: 5
Sample units: 711
alpha: 0.726


The Cronbach's Alpha is greater than 0.7. Hence, the variable can be joint into a unique index, that I called in my paper **Body Image 
Social Comparison (BISC) Index**:

In [ ]:
#7 Generating the new variable “Appsocial” (BISC Index)
BDS$Appsocial <- ((B3$B3_1 + B3$B3_2 + B3$B3_3 +B3$B3_4_r + B3$B3_5 - 5)/20)*100

This new metric variable allows me to do some statistical analysis with some possible predictors. First of all, we can try to predict the degree of social conditioning through the sex, by using an ANOVA model and visualizing the confidence intervals by sex for the variable.

In [ ]:
#8 Priting mean values of the BISC Index (Appsocial) variable for males and females
#loading the package
install.packages("gmodels")
library(gmodels)
#confidence intervals
print("Male CI")
round(ci(na.omit(BDS$Appsocial[BDS$sesso=="M"])),digit=3)
print("Female CI")
round(ci(na.omit(BDS$Appsocial[BDS$sesso=="F"])),digit=3)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘gtools’, ‘gdata’




[1] "Male CI"


Warning message in ci.numeric(na.omit(BDS$Appsocial[BDS$sesso == "M"])):
“No class or unkown class.  Using default calcuation.”


Estimate   CI lower   CI upper Std. Error 
    34.358     31.782     36.934      1.308

[1] "Female CI"


Warning message in ci.numeric(na.omit(BDS$Appsocial[BDS$sesso == "F"])):
“No class or unkown class.  Using default calcuation.”


Estimate   CI lower   CI upper Std. Error 
    42.401     40.588     44.214      0.923

In [ ]:
#9 Analyzing the relationship between sex and BISC Index
genB3 <- aov(BDS$Appsocial~BDS$sesso)
summary(genB3) #sex is signficant

             Df Sum Sq Mean Sq F value   Pr(>F)    
BDS$sesso     1  10616   10616   26.17 4.03e-07 ***
Residuals   709 287602     406                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
33 observations deleted due to missingness

This corroborates the previously hypothesized association between sex and social pressure on the perception of the body. Looking at the mean values, the average tendency to compare one’s own appearance to the one of the other people in social context is higher in females. 

Anyway, it's important to consider other possible features, such as individual traits (more specifically, the Big 5 personality traits), that can play a role in determining the relationship between people and their appearance. Since, from this dataset, Neuroticism, Extraversion and Agreeableness turns out to be associated to sex, (with higher scores related to these traits in women than in men), we will have to consider that while analyzing relationships between personality traits and other features.
Another variable that is considered, since we're dealing with a questionnaire addrssed to students, is the department of origin of the interviewee. This is because different departments can be characterized by differences in gender distribution, economic stratification, competitiveness level, passions, interests and political beliefs of the students, and so, all these variables could act and/or interact with the social pressure on one's own appearance.
A linear model is computed including all these variables.

In [ ]:
#10 Grouping the university attended by the students in a dummy variable
BDS$dipgroup <- BDS$dip
BDS$dipgroup[BDS$dip==1]<-"Hill" #C3A
BDS$dipgroup[BDS$dip==2]<-"Hill" #Biology
BDS$dipgroup[BDS$dip==3]<-"Trento/Rovereto" #Mind/Brain Sciences
BDS$dipgroup[BDS$dip==4]<-"Trento/Rovereto" #Economics
BDS$dipgroup[BDS$dip==5]<-"Hill" #Physics
BDS$dipgroup[BDS$dip==6]<-"Hill" #Civil/Enc/Mech Engineering
BDS$dipgroup[BDS$dip==7]<-"Hill" #Industrial Engineering
BDS$dipgroup[BDS$dip==8]<-"Hill" #Computer Science
BDS$dipgroup[BDS$dip==9]<-"Trento/Rovereto" #Humanities
BDS$dipgroup[BDS$dip==10]<-"Hill" #Mathematics
BDS$dipgroup[BDS$dip==11]<-"Trento/Rovereto" #Psychology
BDS$dipgroup[BDS$dip==12]<-"Trento/Rovereto" #Sociology
BDS$dipgroup[BDS$dip==13]<-"Trento/Rovereto" #Law
BDS$dipgroup[BDS$dip==14]<-"Trento/Rovereto"#International studies
BDS$dipgroup <- as.factor(BDS$dipgroup)
#10.1 Normalizing self esteem scale to have it in a range (0,100):
BDS$self_esteeme<-(BDS$self_esteeme/30)*100
#11 building the linear model to analyze the rel between Big 5 and Appsocial:
Big5applm <- lm(Appsocial~Extraversion+Agreeableness+Conscientiousness+Neuroticism+Openness+sesso+self_esteeme+dipgroup,data=BDS,weights=BDS$peso1)
#12 summarizing
summary(Big5applm)



Call:
lm(formula = Appsocial ~ Extraversion + Agreeableness + Conscientiousness + 
    Neuroticism + Openness + sesso + self_esteeme + dipgroup, 
    data = BDS, weights = BDS$peso1)

Weighted Residuals:
     Min       1Q   Median       3Q      Max 
-167.896  -28.672    1.349   27.180  165.812 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)    
(Intercept)             53.73969    6.37410   8.431  < 2e-16 ***
Extraversion             0.11229    0.03776   2.974  0.00305 ** 
Agreeableness           -0.13689    0.05268  -2.598  0.00957 ** 
Conscientiousness        0.05710    0.04152   1.375  0.16949    
Neuroticism              0.14041    0.04378   3.207  0.00140 ** 
Openness                 0.04014    0.04231   0.949  0.34308    
sessoM                  -7.12510    1.68144  -4.237 2.58e-05 ***
self_esteeme            -0.28009    0.05146  -5.443 7.35e-08 ***
dipgroupTrento/Rovereto -4.69957    1.75956  -2.671  0.00775 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’

Anlayzing the results, it turns out that the inserted personality traits, expcept for Openness and Conscientiousness, are significant in determining the social pressure in the relationship with body in social contexts. The one that seems to be more incisive is Neuroticism (high levels of Neuroticism predict high levels of social pressure on body acceptance). It suggests that feelings of anxiety, apprehension and loleliness stimulate the search for social approval (e.g. by the peer group) and/or the tendency to satisfy body-related stereotypes. As hypothesizable, an even more important role in determining social pressure is played by self-esteem, to the extent that a greater self-esteem predicts low levels of social pressure, presumably because of the absence of the necessity to be dependent by other people to acquire confidence. As stated before, sex is also a significant variable. Furthermore, also the department of study seems to play a role in defining social pressure.

Now, let's see what specifically happens considering the same model for men and women:

In [ ]:
#13 BISC Model for males
Big5appmale <- lm(Appsocial~Extraversion+Agreeableness+Neuroticism+Openness+Conscientiousness+dipgroup+self_esteeme,data=BDS[BDS$sesso=="M",],weights=peso1)
summary(Big5appmale)
#14 Standardized betas
lm.beta(Big5appmale)


Call:
lm(formula = Appsocial ~ Extraversion + Agreeableness + Neuroticism + 
    Openness + Conscientiousness + dipgroup + self_esteeme, data = BDS[BDS$sesso == 
    "M", ], weights = peso1)

Weighted Residuals:
     Min       1Q   Median       3Q      Max 
-174.378  -36.836   -5.087   31.696  148.205 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)    
(Intercept)             48.15907    9.55008   5.043  9.1e-07 ***
Extraversion             0.19010    0.06577   2.890 0.004202 ** 
Agreeableness           -0.34762    0.08924  -3.895 0.000127 ***
Neuroticism              0.19230    0.07426   2.590 0.010199 *  
Openness                 0.08441    0.07424   1.137 0.256701    
Conscientiousness        0.14616    0.07114   2.055 0.041008 *  
dipgroupTrento/Rovereto -4.93216    2.66810  -1.849 0.065761 .  
self_esteeme            -0.31367    0.09134  -3.434 0.000701 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 54


Call:
lm(formula = Appsocial ~ Extraversion + Agreeableness + Neuroticism + 
    Openness + Conscientiousness + dipgroup + self_esteeme, data = BDS[BDS$sesso == 
    "M", ], weights = peso1)

Standardized Coefficients::
            (Intercept)            Extraversion           Agreeableness 
             0.00000000              0.20021656             -0.27456280 
            Neuroticism                Openness       Conscientiousness 
             0.18948551              0.07326233              0.14454133 
dipgroupTrento/Rovereto            self_esteeme 
            -0.11773483             -0.26179996 


In [ ]:
#15 BISC model for females
Big5appfemale <- lm(Appsocial~Extraversion+Agreeableness+Neuroticism+Openness+Conscientiousness+dipgroup+self_esteeme,data=BDS[BDS$sesso=="F",],weights=peso1)
summary(Big5appfemale)
lm.beta(Big5appfemale)


Call:
lm(formula = Appsocial ~ Extraversion + Agreeableness + Neuroticism + 
    Openness + Conscientiousness + dipgroup + self_esteeme, data = BDS[BDS$sesso == 
    "F", ], weights = peso1)

Weighted Residuals:
    Min      1Q  Median      3Q     Max 
-122.57  -25.55    0.50   27.73  101.94 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)    
(Intercept)             51.28319    8.32175   6.163 1.66e-09 ***
Extraversion             0.04236    0.04500   0.941   0.3470    
Agreeableness            0.05652    0.06399   0.883   0.3775    
Neuroticism              0.09822    0.05350   1.836   0.0671 .  
Openness                -0.01425    0.04996  -0.285   0.7756    
Conscientiousness       -0.02715    0.05071  -0.535   0.5927    
dipgroupTrento/Rovereto -3.50502    2.49349  -1.406   0.1606    
self_esteeme            -0.24765    0.05999  -4.128 4.40e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 39.24 on 427


Call:
lm(formula = Appsocial ~ Extraversion + Agreeableness + Neuroticism + 
    Openness + Conscientiousness + dipgroup + self_esteeme, data = BDS[BDS$sesso == 
    "F", ], weights = peso1)

Standardized Coefficients::
            (Intercept)            Extraversion           Agreeableness 
             0.00000000              0.04734643              0.04456336 
            Neuroticism                Openness       Conscientiousness 
             0.09933229             -0.01377997             -0.02744350 
dipgroupTrento/Rovereto            self_esteeme 
            -0.06256090             -0.23266189 


Suprisingly, differently from what happens in males, no one of the Big 5 personality traits is significant in determining how much female are exposed to social pressure in relating to their body. This highlights the presence of a difference in the population of men and women that show certain personality traits. It could be due to different causes able to generate high levels of traits such as Neuroticism in the different genders.

<b>What features predict body care in private contexts?</b>
<BR>
After having considered body care in social contexts, the item B4 is considered, which measures the satisfaction with physical appearance in private contexts.In this case, we are measuring the body perception only taking into account the individual dimension. In fact, in this case, there is no reference to contexts which provide for social interaction, but to the attitude put towards one’s body in a private and intimate context. 
In oder words, this item is taking into account the degree of satisfaction that a person perceives while thinking 
about his/her physical appearance in a private context. I called the concept measured by this item
<b>Private Appearance Satisfaction (PAS) Index</b>

In [ ]:
#16 Analyzing the relationship between sex and body perception in private contexts
BDS$Apppriv<-((BDS$B4-1)/(5-1))*100
B4sex <- table(BDS$sesso,BDS$Apppriv)
chisq.test(B4sex) #independence


	Pearson's Chi-squared test

data:  B4sex
X-squared = 7.5401, df = 4, p-value = 0.11


While in the case of social contexts we observed a significant relationship between sex and relation with one's own appearance, in the case of satisfaction with one's own body in private contexts, there aren't significant relationships.

In [ ]:
#17 building the linear model to analyze the rel between Big 5 and Apppriv (PAS Index):
Big5B4lm <- lm(Apppriv~Extraversion+Agreeableness+Conscientiousness+Neuroticism+Openness+sesso+self_esteeme+dipgroup,data=BDS,weights=BDS$peso1)
#18 summarizing
summary(Big5B4lm)
#19 standardized betas:
lm.beta(Big5B4lm)


Call:
lm(formula = Apppriv ~ Extraversion + Agreeableness + Conscientiousness + 
    Neuroticism + Openness + sesso + self_esteeme + dipgroup, 
    data = BDS, weights = BDS$peso1)

Weighted Residuals:
    Min      1Q  Median      3Q     Max 
-181.46  -29.37    3.39   38.45  155.13 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)             20.502528   6.851860   2.992  0.00287 ** 
Extraversion            -0.029640   0.040246  -0.736  0.46170    
Agreeableness            0.074545   0.056369   1.322  0.18647    
Conscientiousness       -0.005238   0.044619  -0.117  0.90658    
Neuroticism             -0.109885   0.046935  -2.341  0.01951 *  
Openness                -0.021372   0.045438  -0.470  0.63825    
sessoM                  -3.377505   1.803081  -1.873  0.06148 .  
self_esteeme             0.606195   0.055251  10.972  < 2e-16 ***
dipgroupTrento/Rovereto -0.967374   1.886418  -0.513  0.60825    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0


Call:
lm(formula = Apppriv ~ Extraversion + Agreeableness + Conscientiousness + 
    Neuroticism + Openness + sesso + self_esteeme + dipgroup, 
    data = BDS, weights = BDS$peso1)

Standardized Coefficients::
            (Intercept)            Extraversion           Agreeableness 
            0.000000000            -0.027036259             0.050415107 
      Conscientiousness             Neuroticism                Openness 
           -0.004354576            -0.093392081            -0.016512387 
                 sessoM            self_esteeme dipgroupTrento/Rovereto 
           -0.066923271             0.455667013            -0.017793702 


In the case of relationship with appearance in private contexts, the role of Big 5 is totally scaled down. Only Neuroticism remain significant, and with a reduced impact with respect to the one observed in the previous model. Looking at the standardized coefficient, we can see how the most important predictor is of the relationship with body in private context is self-esteem, in a positive verse, taht is, a higher self-esteem increase predicts higher levels of satisfaction with one's body in private contexts.

About the successive items considered (B5), there were 7 questions. Some of these were about the self-appreciation of one's appearance, while others were about the perception of being seen by other people as  attractive. Since these two aspects observed, I hypothesized the presence of 2 indeces within this battery. So I tried to perform parallel analysis, that suggested the presence of only one component among these variables, and then I performed PCA, both without and with rotations (varimax and oblimin rotation), without finding good models. Successfully, I performed Factor Analysis with oblimin rotation adopting a Kaiser normalization, finding two highly correlated factors, which suggested me to consider the possibility to treat all these variables as a unique scale, computing Cronbach's Alpha: 

In [ ]:
#20 B5: selecting variables aimed at analyzing B5 items
B5 <- subset(BDS, select=c(B5_1,B5_2,B5_3,B5_4,B5_5,B5_6,B5_7))
#21 reversing negative items
B5$B5_6_r<- 6-BDS$B5_6
B5$B5_7_r<- 6-BDS$B5_7
#dropping the original negative items:
B5$B5_6 <- NULL
B5$B5_7 <- NULL

In [ ]:
#21 Computing Cronbach's Alpha:
cronbach.alpha(na.omit(B5))


Cronbach's alpha for the 'na.omit(B5)' data-set

Items: 7
Sample units: 700
alpha: 0.894


As presumed, the Cronbach's alpha has been higher, which brought me to build a composite indicator, the **Perceived Body Desirability (PBD)** overall index.

In [ ]:
#22 Building Perceived Body Desirability overall index
BDS$PBD<-((B5$B5_1+B5$B5_2+B5$B5_3+B5$B5_4+B5$B5_5+B5$B5_6_r+B5$B5_7_r-7)/28)*100

Then, the relationship between sex and perceived body desirability is explored:

In [ ]:
#23 Exploring the relationship between sex and B5
B5sex <- aov((BDS$PBD)~BDS$sesso)
summary(B5sex)

             Df Sum Sq Mean Sq F value Pr(>F)
BDS$sesso     1     17    17.4    0.04  0.841
Residuals   698 300113   430.0               
44 observations deleted due to missingness

The model showed absence of significant relationships between sex and perception of body desirability. So far, sex had only a significant direct relationship with social pressure on body care.

In [ ]:
#24 Exploring the relationship between PBD (B5) variable and Big5,B3,B4,sex,dipgroup
B5Big5_<- lm((PBD~Extraversion+Agreeableness+Conscientiousness+Neuroticism+Openness+sesso+Appsocial+self_esteeme+Apppriv+dipgroup),data=BDS,weights=peso1)
summary(B5Big5_)
lm.beta(B5Big5_)


Call:
lm(formula = (PBD ~ Extraversion + Agreeableness + Conscientiousness + 
    Neuroticism + Openness + sesso + Appsocial + self_esteeme + 
    Apppriv + dipgroup), data = BDS, weights = peso1)

Weighted Residuals:
     Min       1Q   Median       3Q      Max 
-144.879  -17.360    0.613   16.218   96.099 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)              2.274142   4.344413   0.523 0.600828    
Extraversion             0.085333   0.024423   3.494 0.000508 ***
Agreeableness           -0.008867   0.034020  -0.261 0.794443    
Conscientiousness       -0.039053   0.026717  -1.462 0.144303    
Neuroticism              0.065842   0.028392   2.319 0.020699 *  
Openness                 0.066858   0.027495   2.432 0.015298 *  
sessoM                  -0.384695   1.099208  -0.350 0.726470    
Appsocial               -0.124289   0.025284  -4.916 1.12e-06 ***
self_esteeme             0.307125   0.036347   8.450  < 2e-16 ***
Apppriv       


Call:
lm(formula = (PBD ~ Extraversion + Agreeableness + Conscientiousness + 
    Neuroticism + Openness + sesso + Appsocial + self_esteeme + 
    Apppriv + dipgroup), data = BDS, weights = peso1)

Standardized Coefficients::
            (Intercept)            Extraversion           Agreeableness 
            0.000000000             0.090714299            -0.007011114 
      Conscientiousness             Neuroticism                Openness 
           -0.038134067             0.065683147             0.059761105 
                 sessoM               Appsocial            self_esteeme 
           -0.008919274            -0.122708935             0.271021568 
                Apppriv dipgroupTrento/Rovereto 
            0.572784725             0.052303269 


In [ ]:
#24.2 Exploring the relationship between PBD (B5) variable and Big5,B3,B4,sex,dipgroup
B5Big5_2<- lm((PBD~Extraversion+Agreeableness+Conscientiousness+Neuroticism+Openness+sesso+self_esteeme+dipgroup),data=BDS,weights=peso1)
summary(B5Big5_2)
lm.beta(B5Big5_2)


Call:
lm(formula = (PBD ~ Extraversion + Agreeableness + Conscientiousness + 
    Neuroticism + Openness + sesso + self_esteeme + dipgroup), 
    data = BDS, weights = peso1)

Weighted Residuals:
    Min      1Q  Median      3Q     Max 
-170.13  -21.87    2.44   27.39  129.87 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)              5.641239   5.391614   1.046   0.2958    
Extraversion             0.057356   0.031806   1.803   0.0718 .  
Agreeableness            0.055313   0.044584   1.241   0.2152    
Conscientiousness       -0.056812   0.035285  -1.610   0.1079    
Neuroticism             -0.009688   0.037172  -0.261   0.7945    
Openness                 0.048714   0.036341   1.340   0.1805    
sessoM                  -1.292369   1.423788  -0.908   0.3644    
self_esteeme             0.642250   0.043676  14.705   <2e-16 ***
dipgroupTrento/Rovereto  2.476704   1.496728   1.655   0.0985 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*


Call:
lm(formula = (PBD ~ Extraversion + Agreeableness + Conscientiousness + 
    Neuroticism + Openness + sesso + self_esteeme + dipgroup), 
    data = BDS, weights = peso1)

Standardized Coefficients::
            (Intercept)            Extraversion           Agreeableness 
            0.000000000             0.061287214             0.043780945 
      Conscientiousness             Neuroticism                Openness 
           -0.055520715            -0.009665716             0.043599519 
                 sessoM            self_esteeme dipgroupTrento/Rovereto 
           -0.030064774             0.567196158             0.053386455 


As for the successive questions, these are based on two typologies: the tipology of male body shape, and the one of female body shape. These are articulated in 5 types:
<p dir="ltr" style="line-height:1.38;margin-top:12pt;margin-bottom:12pt;"><span style="font-size:9pt;font-family:'Times New Roman';color:#244061;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;"><span style="border:none;display:inline-block;overflow:hidden;width:239px;height:172px;"><img src="https://lh5.googleusercontent.com/G0H1slMeO59GKTmKKyBoMYomoaWtrYnPldev7DGmFcUWFoXbepdEt7jd_2moarXHKlCnVa9MOEOyksnEUHWzE49mQQXjKHM7eDQQQPWKcYl0gN3NRKjLN8p70GzAm8EkVpjvAhMg" width="239" height="172"></span></span><span style="font-size:9pt;font-family:'Times New Roman';color:#244061;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">&nbsp;</span></p> <p dir="ltr" style="line-height:1.38;margin-top:0pt;margin-bottom:0pt;"><span style="font-size:9pt;font-family:'Times New Roman';color:#244061;background-color:transparent;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;"></span><span style="font-size:9pt;font-family:'Times New Roman';color:#244061;background-color:transparent;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;"><br></span><span style="font-size:9pt;font-family:'Times New Roman';color:#244061;background-color:transparent;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;"><span style="border:none;display:inline-block;overflow:hidden;width:251px;height:214px;"><img src="https://lh4.googleusercontent.com/sbLbP9hUpEn8Y_YMez5G2XBRitLEb75_ksvCN49dxoySSW0xiYJr9RNG0-iegCeDaeo7_9ZlFWiho8gxfpcAN6_yen5dmmajh1QpOAUwTQwxiH6GOjTs6OXT6hwbRHDGSRM0vqn3" width="251" height="214"></span></span></p>


In this case, some questions are about the identification with a certain type of body within a typology composed of five possible declinations, as well as the appreciation or the perception about how the others perceive certain kinds of body. To begin with, considering the first two specific questions (the perceived body shape and the desired body shape), they have been joint in a unique dummy variable (Body shape acceptance) that indicates if the perceived body shape of an individual corresponds to its desired body shape or not. The same has been done for the first and the fourth questions, creating a dummy variables that indicates if a person has or not a body shape that they consider as desired by the other sex.

In [ ]:
#25 creating the Body Shape Acceptance variable
BDS$BDSAcceptance[BDS$sesso=="F"]<- as.numeric(BDS$B6f_1_1[BDS$sesso=="F"]==BDS$B6f_2_1[BDS$sesso=="F"])
BDS$BDSAcceptance[BDS$sesso=="M"]<- as.numeric(BDS$B6m_1_1[BDS$sesso=="M"]==BDS$B6m_2_1[BDS$sesso=="M"])

In [ ]:
#26 Createing the Other  Sex Perceived Attraction 
#(Perceived Body Shape Attraction - PBSA) variable:
BDS$PBSA[BDS$sesso=="F"]<- as.numeric(BDS$B6f_1_1[BDS$sesso=="F"]==BDS$B6f_4_1[BDS$sesso=="F"])
BDS$PBSA[BDS$sesso=="M"]<- as.numeric(BDS$B6m_1_1[BDS$sesso=="M"]==BDS$B6m_5_1[BDS$sesso=="M"])

Then I computed the relationship between body shape acceptance in males and females, hypothesizing a higher body acceptance in males than in females.

In [ ]:
#27 creating the table
BDSAsex <- table(BDS$sesso,BDS$BDSAcceptance)
#visualizing the table
BDSAsex
round(prop.table(BDSAsex,1),digit=3)*100
#chi-square test
chisq.test(BDSAsex)

   
      0   1
  F 302 126
  M 197  54

   
       0    1
  F 70.6 29.4
  M 78.5 21.5


	Pearson's Chi-squared test with Yates' continuity correction

data:  BDSAsex
X-squared = 4.7022, df = 1, p-value = 0.03012


Surprisingly he table above shows the opposite thing: in females, a significantly higher proportion of people who accept their body has been found. This may be due to the fact that the population of male students is more composed than women of people with a stereotipically considered less attractive body shape.

In [ ]:
#28 visualize the big glm logit model with BDSAcceptance as our y
install.packages("DescTools")
library(DescTools)

#28.1 Creating the dummy BA/MA
BDS$coursetype <- NA 
BDS$coursetype[BDS$tipcor==5]<-NA #dropping people in a "ciclo unico" course, because it gives no inofrmation about the year
BDS$coursetype[BDS$tipcor==2]<-"MA" # MA people
BDS$coursetype[BDS$tipcor==1]<-"BA" #BA people
#28.2 model
BSAmodel <-glm(BDS$BDSAcceptance~BDS$Neuroticism+BDS$Conscientiousness+BDS$Openness+BDS$Agreeableness+BDS$Extraversion+BDS$sesso+BDS$self_esteeme+BDS$Appsocial+BDS$Apppriv+BDS$dipgroup+BDS$PBD+BDS$coursetype, family=binomial("logit"))
summary(BSAmodel)
round(PseudoR2(BSAmodel, which="McKelveyZavoina"),digit=2)
lm.beta(BSAmodel)
#Computing the pseudo-R2 to understand what is the explained variation:


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘proxy’, ‘rootSolve’, ‘e1071’, ‘lmom’, ‘Exact’, ‘gld’


Registered S3 method overwritten by 'DescTools':
  method         from 
  reorder.factor gdata


Attaching package: ‘DescTools’


The following objects are masked from ‘package:psych’:

    AUC, ICC, SD





Call:
glm(formula = BDS$BDSAcceptance ~ BDS$Neuroticism + BDS$Conscientiousness + 
    BDS$Openness + BDS$Agreeableness + BDS$Extraversion + BDS$sesso + 
    BDS$self_esteeme + BDS$Appsocial + BDS$Apppriv + BDS$dipgroup + 
    BDS$PBD + BDS$coursetype, family = binomial("logit"))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8414  -0.7325  -0.4273  -0.1162   2.6903  

Coefficients:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -3.486522   1.056729  -3.299 0.000969 ***
BDS$Neuroticism             -0.004965   0.006495  -0.764 0.444670    
BDS$Conscientiousness        0.008539   0.006392   1.336 0.181589    
BDS$Openness                -0.006342   0.006469  -0.980 0.326850    
BDS$Agreeableness            0.002913   0.008377   0.348 0.728049    
BDS$Extraversion            -0.004405   0.005839  -0.754 0.450583    
BDS$sessoM                  -0.238920   0.271123  -0.881 0.378198    
BDS$self_esteeme            -

McKelveyZavoina 
           0.36


Call:
glm(formula = BDS$BDSAcceptance ~ BDS$Neuroticism + BDS$Conscientiousness + 
    BDS$Openness + BDS$Agreeableness + BDS$Extraversion + BDS$sesso + 
    BDS$self_esteeme + BDS$Appsocial + BDS$Apppriv + BDS$dipgroup + 
    BDS$PBD + BDS$coursetype, family = binomial("logit"))

Standardized Coefficients::
                (Intercept)             BDS$Neuroticism 
                  0.0000000                  -0.2407775 
      BDS$Conscientiousness                BDS$Openness 
                  0.4097890                  -0.2713809 
          BDS$Agreeableness            BDS$Extraversion 
                  0.1090658                  -0.2258636 
                 BDS$sessoM            BDS$self_esteeme 
                 -0.2731862                  -0.4873436 
              BDS$Appsocial                 BDS$Apppriv 
                 -0.7514173                   0.2884655 
BDS$dipgroupTrento/Rovereto                     BDS$PBD 
                  0.1397895                   2.8053929 
     

In [ ]:
#29 same model without body-related composite indicators
BSA2model <-glm(BDS$BDSAcceptance~BDS$Neuroticism+BDS$Conscientiousness+BDS$Openness+BDS$Agreeableness+BDS$Extraversion+BDS$sesso+BDS$self_esteeme+BDS$dipgroup+BDS$coursetype, family=binomial("logit"))
summary(BSA2model)
round(PseudoR2(BSA2model, which="McKelveyZavoina"),digit=2)
lm.beta(BSA2model)
#Computing the pseudo-R2 to understand what is the explained variation:



Call:
glm(formula = BDS$BDSAcceptance ~ BDS$Neuroticism + BDS$Conscientiousness + 
    BDS$Openness + BDS$Agreeableness + BDS$Extraversion + BDS$sesso + 
    BDS$self_esteeme + BDS$dipgroup + BDS$coursetype, family = binomial("logit"))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2432  -0.7765  -0.6377  -0.4245   2.2205  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -2.8342946  0.8904679  -3.183 0.001458 ** 
BDS$Neuroticism             -0.0027586  0.0057331  -0.481 0.630397    
BDS$Conscientiousness        0.0042657  0.0056544   0.754 0.450612    
BDS$Openness                -0.0015253  0.0058642  -0.260 0.794777    
BDS$Agreeableness            0.0002275  0.0072949   0.031 0.975119    
BDS$Extraversion            -0.0008144  0.0051133  -0.159 0.873462    
BDS$sessoM                  -0.1531238  0.2427429  -0.631 0.528167    
BDS$self_esteeme             0.0240241  0.0072617   3.308 0.000938

McKelveyZavoina 
           0.07


Call:
glm(formula = BDS$BDSAcceptance ~ BDS$Neuroticism + BDS$Conscientiousness + 
    BDS$Openness + BDS$Agreeableness + BDS$Extraversion + BDS$sesso + 
    BDS$self_esteeme + BDS$dipgroup + BDS$coursetype, family = binomial("logit"))

Standardized Coefficients::
                (Intercept)             BDS$Neuroticism 
                0.000000000                -0.133992525 
      BDS$Conscientiousness                BDS$Openness 
                0.204210459                -0.065265627 
          BDS$Agreeableness            BDS$Extraversion 
                0.008518558                -0.041955725 
                 BDS$sessoM            BDS$self_esteeme 
               -0.175645100                 1.027376924 
BDS$dipgroupTrento/Rovereto            BDS$coursetypeMA 
                0.207085067                 0.122078176 


In [ ]:
#30 model with only the y and Perceived Body desirability
nooomodel <-glm(BDS$BDSAcceptance~BDS$PBD, family=binomial("logit"))
summary(nooomodel)
round(PseudoR2(nooomodel, which="McKelveyZavoina"),digit=2)



Call:
glm(formula = BDS$BDSAcceptance ~ BDS$PBD, family = binomial("logit"))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8281  -0.7416  -0.4500   0.7824   2.7519  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -4.63450    0.40221 -11.522   <2e-16 ***
BDS$PBD      0.06097    0.00617   9.881   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 780.85  on 674  degrees of freedom
Residual deviance: 642.63  on 673  degrees of freedom
  (69 observations deleted due to missingness)
AIC: 646.63

Number of Fisher Scoring iterations: 5


McKelveyZavoina 
           0.33

The results are problematic. Predicting wheter a person will accept or not their body seems to be complicated. In fact, apart from the type of course (BA/MA, which brings to a differentiaton by age of the sampled individuals), the only significant predictors are the perceived body desirability and the social pressure on body care. Hypothesizing that this may be due to the fact that these two variables could measure dimensions of the same concept mesarured by the y of this model, I tried to drop these variables from the model. However, it brought limited results and the only self-esteem as a predictor (along with a small pseufo-R^2).

Successively, a model is computed using the other dummy variable, that detects wheter a person considers its body shape as the one appreciated by the other sex (Other Sex Perceived Attraction - OSPA)

In [ ]:
#31.2 PBSA (OBSA) Model
PBSAmodel <-glm(BDS$PBSA~BDS$Neuroticism+BDS$Conscientiousness+BDS$Openness+BDS$Agreeableness+BDS$Extraversion+BDS$sesso+BDS$Appsocial+BDS$Apppriv+BDS$self_esteeme+BDS$dipgroup+BDS$PBD+BDS$coursetype, family=binomial("logit"))
summary(PBSAmodel)
round(PseudoR2(PBSAmodel, which="McKelveyZavoina"),digit=2)


Call:
glm(formula = BDS$PBSA ~ BDS$Neuroticism + BDS$Conscientiousness + 
    BDS$Openness + BDS$Agreeableness + BDS$Extraversion + BDS$sesso + 
    BDS$Appsocial + BDS$Apppriv + BDS$self_esteeme + BDS$dipgroup + 
    BDS$PBD + BDS$coursetype, family = binomial("logit"))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1898  -0.5695  -0.4375  -0.2843   2.8315  

Coefficients:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -4.378176   1.211107  -3.615 0.000300 ***
BDS$Neuroticism             -0.002089   0.007442  -0.281 0.778964    
BDS$Conscientiousness       -0.001371   0.007405  -0.185 0.853094    
BDS$Openness                -0.006844   0.007598  -0.901 0.367701    
BDS$Agreeableness            0.002299   0.009686   0.237 0.812405    
BDS$Extraversion             0.002404   0.006934   0.347 0.728802    
BDS$sessoM                   0.292469   0.315712   0.926 0.354249    
BDS$Appsocial                0.007222 

McKelveyZavoina 
           0.18

In [ ]:
#32
PBSAmodel1 <-glm(BDS$PBSA~BDS$PBD, family=binomial("logit"))
summary(PBSAmodel1)
round(PseudoR2(PBSAmodel1, which="McKelveyZavoina"),digit=2)


Call:
glm(formula = BDS$PBSA ~ BDS$PBD, family = binomial("logit"))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1645  -0.5976  -0.4429  -0.2781   2.7910  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -4.514844   0.457927  -9.859  < 2e-16 ***
BDS$PBD      0.044843   0.006766   6.628 3.41e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 543.38  on 669  degrees of freedom
Residual deviance: 489.75  on 668  degrees of freedom
  (74 observations deleted due to missingness)
AIC: 493.75

Number of Fisher Scoring iterations: 5


McKelveyZavoina 
           0.21

In [ ]:
#33
PBSAmodel2 <-glm(BDS$PBSA~BDS$Neuroticism+BDS$Conscientiousness+BDS$Openness+BDS$Agreeableness+BDS$Extraversion+BDS$self_esteeme+BDS$sesso+BDS$dipgroup+BDS$coursetype, family=binomial("logit"))
summary(PBSAmodel2)
round(PseudoR2(PBSAmodel2, which="McKelveyZavoina"),digit=2)


Call:
glm(formula = BDS$PBSA ~ BDS$Neuroticism + BDS$Conscientiousness + 
    BDS$Openness + BDS$Agreeableness + BDS$Extraversion + BDS$self_esteeme + 
    BDS$sesso + BDS$dipgroup + BDS$coursetype, family = binomial("logit"))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-0.8346  -0.5683  -0.4779  -0.3821   2.4880  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)   
(Intercept)                 -3.6441367  1.1134024  -3.273  0.00106 **
BDS$Neuroticism              0.0002989  0.0071003   0.042  0.96642   
BDS$Conscientiousness       -0.0014775  0.0071220  -0.207  0.83565   
BDS$Openness                -0.0032255  0.0074349  -0.434  0.66441   
BDS$Agreeableness            0.0021107  0.0092545   0.228  0.81959   
BDS$Extraversion             0.0069301  0.0065256   1.062  0.28824   
BDS$self_esteeme             0.0236841  0.0093721   2.527  0.01150 * 
BDS$sessoM                   0.2027251  0.3009033   0.674  0.50049   
BDS$dipgroupT

McKelveyZavoina 
           0.07

In this case, the results are even worse, with a lower explained variability and perceived body desirabiity as the only predictor of PBSA. The only variable that, erasing body-relating ideces from the predictors, emerges, is always self-esteem, but with a very lower explained variability with respect to the Other sex perceived attraction.

<b>Latent class modelling based on B6 items </b>
<BR>
However, considering other possible intersections between the specific perceived and desired or considered as attractive body shapes, I decided to try to classify the dataset according to possible latent class based on B6 items. In both the cases, looking at Bayesian Information Criterion, I found that the best model was the one with 3 classes, while the AIC tended to converge to excessively large numbers of classes, which brought me to follow the BIC in choosing the number of classes.

In [ ]:
#34 Preparing variables for the LCA
install.packages("poLCA")
library(poLCA)
xs2<-cbind(B6f_1_1,B6f_2_1,B6f_3_1,B6f_4_1,B6f_5_1)~1

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘scatterplot3d’


Loading required package: scatterplot3d



Let's explore the latent classes for females: <p dir="ltr" style="line-height:1.38;margin-top:0pt;margin-bottom:0pt;"><span style="font-size:9pt;font-family:'Times New Roman';color:#244061;background-color:transparent;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;"></span><span style="font-size:9pt;font-family:'Times New Roman';color:#244061;background-color:transparent;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;"><br></span><span style="font-size:9pt;font-family:'Times New Roman';color:#244061;background-color:transparent;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;"><span style="border:none;display:inline-block;overflow:hidden;width:251px;height:214px;"><img src="https://lh4.googleusercontent.com/sbLbP9hUpEn8Y_YMez5G2XBRitLEb75_ksvCN49dxoySSW0xiYJr9RNG0-iegCeDaeo7_9ZlFWiho8gxfpcAN6_yen5dmmajh1QpOAUwTQwxiH6GOjTs6OXT6hwbRHDGSRM0vqn3" width="251" height="214"></span></span></p>


In [ ]:
#35 exploring the best model for females
set.seed(10)
lca2<-poLCA(xs2, BDS, nclass = 3, maxiter = 1500,graphs = FALSE)

Conditional item response (column) probabilities,
 by outcome variable, for each class (row) 
 
$B6f_1_1
           Pr(1)  Pr(2)  Pr(3)  Pr(4)  Pr(5)
class 1:  0.6948 0.0000 0.0545 0.0000 0.2507
class 2:  0.0000 0.0000 0.0000 0.9658 0.0342
class 3:  0.0000 0.3561 0.5365 0.1003 0.0071

$B6f_2_1
           Pr(1)  Pr(2)  Pr(3)  Pr(4)
class 1:  0.5030 0.1829 0.1597 0.1544
class 2:  0.4522 0.2000 0.1924 0.1554
class 3:  0.2302 0.4286 0.2084 0.1328

$B6f_3_1
           Pr(1)  Pr(2)  Pr(3)  Pr(4)  Pr(5)
class 1:  0.6166 0.0231 0.0364 0.0890 0.2349
class 2:  0.0367 0.0005 0.0972 0.8083 0.0573
class 3:  0.0000 0.4056 0.5382 0.0562 0.0000

$B6f_4_1
           Pr(1)  Pr(2)  Pr(3)  Pr(4)  Pr(5)
class 1:  0.3884 0.1993 0.1299 0.2825 0.0000
class 2:  0.6960 0.1363 0.0830 0.0775 0.0073
class 3:  0.7104 0.1276 0.0319 0.1301 0.0000

$B6f_5_1
           Pr(1)  Pr(2)  Pr(3)  Pr(4)  Pr(5)
class 1:  0.2108 0.4493 0.1968 0.1359 0.0071
class 2:  0.3581 0.3538 0.2363 0.0445 0.0073
class 3:  0.2607 0.4087 0.15

In [ ]:
#36 doing the same for males
xm2<-cbind(B6m_1_1,B6m_2_1,B6m_3_1,B6m_4_1,B6m_5_1)~1

<p dir="ltr" style="line-height:1.38;margin-top:12pt;margin-bottom:12pt;"><span style="font-size:9pt;font-family:'Times New Roman';color:#244061;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;"><span style="border:none;display:inline-block;overflow:hidden;width:239px;height:172px;"><img src="https://lh5.googleusercontent.com/G0H1slMeO59GKTmKKyBoMYomoaWtrYnPldev7DGmFcUWFoXbepdEt7jd_2moarXHKlCnVa9MOEOyksnEUHWzE49mQQXjKHM7eDQQQPWKcYl0gN3NRKjLN8p70GzAm8EkVpjvAhMg" width="239" height="172"></span></span><span style="font-size:9pt;font-family:'Times New Roman';color:#244061;background-color:#ffffff;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">&nbsp;</span></p> 

In [ ]:
#37 exploring the best model for males
set.seed(11)
lca2m<-poLCA(xm2, BDS, nclass = 3, maxiter = 1500,graphs = FALSE)

Conditional item response (column) probabilities,
 by outcome variable, for each class (row) 
 
$B6m_1_1
           Pr(1)  Pr(2)  Pr(3)  Pr(4)  Pr(5)
class 1:  0.9533 0.0467 0.0000 0.0000 0.0000
class 2:  0.0000 0.0000 0.0000 0.6683 0.3317
class 3:  0.0396 0.0077 0.7857 0.1670 0.0000

$B6m_2_1
           Pr(1)  Pr(2)  Pr(3)  Pr(4)  Pr(5)
class 1:  0.5626 0.4374 0.0000 0.0000 0.0000
class 2:  0.4741 0.0753 0.3754 0.0753 0.0000
class 3:  0.7892 0.0559 0.1318 0.0155 0.0077

$B6m_3_1
           Pr(1) Pr(2)  Pr(3)  Pr(4)  Pr(5)
class 1:  0.8088  0.07 0.0979 0.0000 0.0233
class 2:  0.0012  0.00 0.0000 0.7726 0.2262
class 3:  0.0559  0.00 0.9152 0.0290 0.0000

$B6m_4_1
           Pr(1)  Pr(2)  Pr(3)  Pr(4)  Pr(5)
class 1:  0.3143 0.6087 0.0537 0.0000 0.0233
class 2:  0.5230 0.3720 0.0749 0.0301 0.0000
class 3:  0.4687 0.4178 0.0518 0.0308 0.0308

$B6m_5_1
           Pr(1)  Pr(2)  Pr(3)  Pr(4)  Pr(5)
class 1:  0.5472 0.4528 0.0000 0.0000 0.0000
class 2:  0.6855 0.1491 0.1051 0.0452 0.0151
clas

After that, I classified each interviewee associating to themselves the respective attributed class by the Latent Class algorithm, with the goal to use the latent class as a predictor in predictive models about other variables.

In [ ]:
#38 Adding the vairable "latent class female"
lclassf<- lca2$predclass
BDS$lclassf<-NA
BDS[complete.cases(BDS[,86:90]),]$lclassf<-lclassf

In [ ]:
#39 Adding the vairable "latent class male"
lclassm <- lca2m$predclass
BDS$lclassm<-NA
BDS[complete.cases(BDS[,91:95]),]$lclassm<-lclassm

I fitted a model to try to predict the presence of tattoo and piercing according to the previously used variables, in addition to the latent classes, in orther to see if there are typologies composed of different types of males and females, that, according to certain body shapes as desired, appreciated and source of identification, tend to be more willing to modify their body:

In [ ]:
#40 Creating a dummy that takes 0 if the person has neither tattoos nor piercings and 1 otherwise
BDS$tatpier[BDS$B13==1]<-0
BDS$tatpier[BDS$B13!=1]<-1

In [ ]:
tatpmodello1<-glm((tatpier~Neuroticism+Conscientiousness+Extraversion+Openness+Agreeableness+self_esteeme+Appsocial+Apppriv+PBD+sesso),data=BDS,family=binomial("logit"))
summary(tatpmodello1)
round(PseudoR2(tatpmodello1, which="McKelveyZavoina"),digit=2)


Call:
glm(formula = (tatpier ~ Neuroticism + Conscientiousness + Extraversion + 
    Openness + Agreeableness + self_esteeme + Appsocial + Apppriv + 
    PBD + sesso), family = binomial("logit"), data = BDS)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.3949  -0.9055  -0.5083   1.1566   2.3652  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)       -1.405618   0.844658  -1.664  0.09609 .  
Neuroticism        0.008751   0.005436   1.610  0.10743    
Conscientiousness  0.009040   0.005169   1.749  0.08032 .  
Extraversion       0.008642   0.004743   1.822  0.06844 .  
Openness           0.003601   0.005284   0.681  0.49557    
Agreeableness     -0.004007   0.006557  -0.611  0.54113    
self_esteeme      -0.010500   0.007297  -1.439  0.15016    
Appsocial         -0.005223   0.005041  -1.036  0.30017    
Apppriv           -0.014117   0.006102  -2.314  0.02069 *  
PBD                0.021384   0.007693   2.780  0.00544 ** 
se

McKelveyZavoina 
            0.2

In [ ]:
#41
tatpmodf<-glm((tatpier~Neuroticism+Conscientiousness+Extraversion+Openness+Agreeableness+self_esteeme+Appsocial+Apppriv+PBD+lclassf),data=BDS[BDS$sesso=="F",],family=binomial("logit"))
summary(tatpmodf)
round(PseudoR2(tatpmodf, which="McKelveyZavoina"),digit=2)


Call:
glm(formula = (tatpier ~ Neuroticism + Conscientiousness + Extraversion + 
    Openness + Agreeableness + self_esteeme + Appsocial + Apppriv + 
    PBD + lclassf), family = binomial("logit"), data = BDS[BDS$sesso == 
    "F", ])

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.4728  -1.0047  -0.7941   1.2366   1.8114  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)   
(Intercept)       -1.127700   1.062266  -1.062  0.28842   
Neuroticism        0.006398   0.006436   0.994  0.32018   
Conscientiousness  0.008918   0.006133   1.454  0.14591   
Extraversion       0.009618   0.005401   1.781  0.07495 . 
Openness           0.004001   0.006277   0.637  0.52387   
Agreeableness     -0.004491   0.007831  -0.573  0.56633   
self_esteeme      -0.010441   0.008754  -1.193  0.23300   
Appsocial         -0.002205   0.005975  -0.369  0.71212   
Apppriv           -0.015315   0.007521  -2.036  0.04171 * 
PBD                0.024992   0.009445   2.64

McKelveyZavoina 
           0.07

In [ ]:
#42
tatpmodm<-glm((tatpier~Neuroticism+Conscientiousness+Extraversion+Openness+Agreeableness+self_esteeme+Appsocial+Apppriv+PBD+lclassm),data=BDS[BDS$sesso=="M",],family=binomial("logit"))
summary(tatpmodm)
round(PseudoR2(tatpmodm, which="McKelveyZavoina"),digit=2)


Call:
glm(formula = (tatpier ~ Neuroticism + Conscientiousness + Extraversion + 
    Openness + Agreeableness + self_esteeme + Appsocial + Apppriv + 
    PBD + lclassm), family = binomial("logit"), data = BDS[BDS$sesso == 
    "M", ])

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-0.9550  -0.5384  -0.4193  -0.3153   2.4682  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)  
(Intercept)       -2.426415   1.842945  -1.317   0.1880  
Neuroticism        0.019431   0.012104   1.605   0.1084  
Conscientiousness  0.011306   0.011548   0.979   0.3276  
Extraversion       0.008033   0.012470   0.644   0.5195  
Openness           0.004742   0.012254   0.387   0.6988  
Agreeableness     -0.010430   0.015728  -0.663   0.5072  
self_esteeme      -0.008235   0.015364  -0.536   0.5919  
Appsocial         -0.027261   0.012119  -2.249   0.0245 *
Apppriv           -0.004802   0.012476  -0.385   0.7003  
PBD                0.013207   0.015185   0.870   0.3844

McKelveyZavoina 
           0.13

The results didn't show good models when dividing the female data from male data. We can anyway see as the perceived body desirability seems to play a role, as a predictor, only in females.

<b>Latent classes exploration</b>
<BR>
I explored the specific features of each generated class:
<BR>
<b>Male latent classes</b>

In [ ]:
#43 Latent classes exploration
round(ci(na.omit(BDS$BDSAcceptance[BDS$lclassm==1])),digit=3)*100

Warning message in ci.numeric(na.omit(BDS$BDSAcceptance[BDS$lclassm == 1])):
“No class or unkown class.  Using default calcuation.”


Estimate   CI lower   CI upper Std. Error 
      60.5       45.2       75.7        7.5

This first class is characterized by a higher proportion of people that accept their body. As visible in the previously generated tables, these people have, almost totally, recognized themselves with the body picture 1.

In [ ]:
round(ci(na.omit(BDS$BDSAcceptance[BDS$lclassm==2])),digit=3)*100

Warning message in ci.numeric(na.omit(BDS$BDSAcceptance[BDS$lclassm == 2])):
“No class or unkown class.  Using default calcuation.”


Estimate   CI lower   CI upper Std. Error 
       6.1        0.1       12.0        3.0

This second class is characterized by a considerably low proportion of people that accept their body. As visible in the previously generated tables, the majotiy of these people have, recognized themselves with the body picture 4 or 5.

In [ ]:
round(ci(na.omit(BDS$BDSAcceptance[BDS$lclassm==3])),digit=3)*100

Warning message in ci.numeric(na.omit(BDS$BDSAcceptance[BDS$lclassm == 3])):
“No class or unkown class.  Using default calcuation.”


Estimate   CI lower   CI upper Std. Error 
      14.0        7.9       20.0        3.1

The third class is in a middle way condition, even if near to the class 2. As visible in the previously generated tables, the majotiy of these people identify themselves with the body shape 3, even if there is a discrete minority in the body shape 4.

<b>Female latent classes</b>

In [ ]:
round(ci(na.omit(BDS$BDSAcceptance[BDS$lclassf==1])),digit=3)*100

Warning message in ci.numeric(na.omit(BDS$BDSAcceptance[BDS$lclassf == 1])):
“No class or unkown class.  Using default calcuation.”


Estimate   CI lower   CI upper Std. Error 
      36.6       28.6       44.6        4.1

This class is mainly represented by the identified body shape 1, although with a large minority from the body shape 5. It is the one with the highest proportion of body accpetance.

In [ ]:
round(ci(na.omit(BDS$BDSAcceptance[BDS$lclassf==2])),digit=3)*100

Warning message in ci.numeric(na.omit(BDS$BDSAcceptance[BDS$lclassf == 2])):
“No class or unkown class.  Using default calcuation.”


Estimate   CI lower   CI upper Std. Error 
      15.2        9.1       21.3        3.1

The large majority of girls from this class identify themselves with the body shape 4. It is the class with the lowest body acceptance proportion.

In [ ]:
round(ci(na.omit(BDS$BDSAcceptance[BDS$lclassf==3])),digit=3)*100

Warning message in ci.numeric(na.omit(BDS$BDSAcceptance[BDS$lclassf == 3])):
“No class or unkown class.  Using default calcuation.”


Estimate   CI lower   CI upper Std. Error 
      36.1       27.8       44.4        4.2

In the middle way, we have the class 3, mostly represented by the figure 3, but with a relevant proportion from classes 2 and 4. As for the body acceptance, it has similar levels to the class 1.

In [ ]:
round(ci(na.omit(BDS$BDSAcceptance[BDS$sesso=="M"])),digit=3)*100

Warning message in ci.numeric(na.omit(BDS$BDSAcceptance[BDS$sesso == "M"])):
“No class or unkown class.  Using default calcuation.”


Estimate   CI lower   CI upper Std. Error 
      21.5       16.4       26.6        2.6

In [ ]:
round(ci(na.omit(BDS$BDSAcceptance[BDS$sesso=="F"])),digit=3)*100

Warning message in ci.numeric(na.omit(BDS$BDSAcceptance[BDS$sesso == "F"])):
“No class or unkown class.  Using default calcuation.”


Estimate   CI lower   CI upper Std. Error 
      29.4       25.1       33.8        2.2